In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass

In [2]:
loss_functions = {
    "Huber":nn.HuberLoss(),
    "MSE":nn.MSELoss(),
    "L1":nn.SmoothL1Loss(reduction='sum', beta=0.5),
    "BCE":nn.CrossEntropyLoss(),
    "Custom":custom_MSE,
    "KL":nn.KLDivLoss(reduction='batchmean')
}

NameError: name 'custom_MSE' is not defined

In [ ]:
huber = loss_functions["Huber"]


In [3]:
def loss_tests(output,target):
    huber = nn.HuberLoss(),
    mse = nn.MSELoss(),
    l1 = nn.SmoothL1Loss(reduction='sum', beta=0.5),
    bce = nn.CrossEntropyLoss(),
    Custom = custom_MSE,
    kl = nn.KLDivLoss(reduction='batchmean')

    print(f"huber = {huber(output,target)}")
    print(f"l1 = {l1(output,target)}")
    print(f"MSE = {mse(output,target)}")
    print(f"bce = {bce(output,target)}")
    print(f"KL = {kl(output,target)}")

In [4]:
sftmax = torch.nn.Softmax(dim=1)
loss_tests(sftmax(x), sftmax(y))

NameError: name 'x' is not defined

In [5]:
def custom_MSE(output, target):
    print(target.float())
    sorts_t = torch.argsort(target)
    sorts_o = torch.argsort(output)
    print(sorts_t)
    print(sorts_o)
    out = sorts_t.narrow(1,0,3)
    print(out)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    print(ohe)
    out_first3 = ohe*output
    target_ohe = ohe*target
    print(f"{out_first3=},\n{target_ohe=}")
    print(f"delta = {abs(out_first3-target_ohe)+1}")
    print(f"delta 2 = {abs(out_first3-target_ohe)**2+1}")
    loss = torch.mean((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print("h",((abs(out_first3-target_ohe)*10)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    

In [6]:
def custom_huber(output, target):
    print(target.float())
    sorts_t = torch.argsort(target)
    sorts_o = torch.argsort(output)
    print(sorts_t)
    print(sorts_o)
    out = sorts_t.narrow(1,0,3)
    print(out)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    print(ohe)
    out_first3 = ohe*output
    target_ohe = ohe*target
    print(f"{out_first3=},\n{target_ohe=}")
    print(f"delta = {0.5*(out_first3-target_ohe)**2/0.2}")
    print(f"delta 2 = {abs(out_first3-target_ohe)**2+1}")
    loss = torch.mean((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print("h",((abs(out_first3-target_ohe)*10)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss

In [7]:
def custom_KL(output, target):
    print(target.float())
    sorts_t = torch.argsort(target)
    sorts_o = torch.argsort(output)
    target100 = target*100
    output100 = output*100
    print(sorts_t)
    print(sorts_o)
    out = sorts_t.narrow(1,0,3)
    print(out)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    print(ohe)
    out_first3 = ohe*output
    target_ohe = ohe*target
    print(f"{out_first3=},\n{target_ohe=}")

    print(f"delta = {abs(out_first3-target_ohe)+1}")

    print(f"reg log deltas = {target100*(target100.log()-output100.log())}")

    print(f"log deltas = {(target_ohe+1)*((target_ohe+1).log()-(out_first3+1).log())}")

    loss = torch.mean((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))

    print((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))

    print("h",((abs(out_first3-target_ohe)*10)**2))

    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    

In [8]:
def custom_order(output, target):
    print(f"{target}\n{output}")
    sorts_t = torch.argsort(target)
    sorts_o = torch.argsort(output)
    print(sorts_t)
    print(sorts_o)
    out = sorts_t.narrow(1,0,3)
    print(out)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    print(ohe)
    out_first3 = ohe*output
    target_ohe = ohe*target
    print(f"{out_first3=},\n{target_ohe=}")
    print(f"delta = {abs(out_first3-target_ohe)+1}")
    print(f"delta 2 = {abs(out_first3-target_ohe)**2+1}")
    loss = torch.mean((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    print("h",((abs(out_first3-target_ohe)*10)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss

In [9]:
x = torch.tensor([[7.0,2.0,6.0,8.0,1.0,3.0,4.0,5.0],[8.0,2.0,4.0,6.0,5.0,3.0,1.0,7.0]])
y = torch.tensor([[8.0,2.0,4.0,6.0,5.0,3.0,1.0,7.0],[7.0,2.0,6.0,8.0,1.0,3.0,4.0,5.0]])
x_idx = custom_KL(sftmax(x),sftmax(y))
x_idx

tensor([[6.3233e-01, 1.5674e-03, 1.1582e-02, 8.5577e-02, 3.1482e-02, 4.2606e-03,
         5.7661e-04, 2.3262e-01],
        [2.3262e-01, 1.5674e-03, 8.5577e-02, 6.3233e-01, 5.7661e-04, 4.2606e-03,
         1.1582e-02, 3.1482e-02]])
tensor([[6, 1, 5, 2, 4, 3, 7, 0],
        [4, 1, 5, 6, 7, 2, 0, 3]])
tensor([[4, 1, 5, 6, 7, 2, 0, 3],
        [6, 1, 5, 2, 4, 3, 7, 0]])
tensor([[6, 1, 5],
        [4, 1, 5]])
tensor([[0, 1, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 1, 1, 0, 0]])
out_first3=tensor([[0.0000, 0.0016, 0.0000, 0.0000, 0.0000, 0.0043, 0.0116, 0.0000],
        [0.0000, 0.0016, 0.0000, 0.0000, 0.0315, 0.0043, 0.0000, 0.0000]]),
target_ohe=tensor([[0.0000, 0.0016, 0.0000, 0.0000, 0.0000, 0.0043, 0.0006, 0.0000],
        [0.0000, 0.0016, 0.0000, 0.0000, 0.0006, 0.0043, 0.0000, 0.0000]])
delta = tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0110, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0309, 1.0000, 1.0000, 1.0000]])
reg log deltas = tensor([[ 63.2333,   0.0000,

tensor(1.0053)

In [ ]:
def loss_customiser(loss_func,x,y):
    sorts_t = torch.argsort(sftmax(y))+1
    sorts_factor = 1/sorts_t
    print(sorts_factor)
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)
    loss_ohe = loss*ohe
    print(f"{sorts_t}\n{out}\n{ohe}\n{loss}\n{loss_ohe}")
    return (torch.mean(loss),torch.mean(loss_ohe))

In [30]:
def loss_customiser(loss_func,x,y):
    sorts_t = torch.argsort(sftmax(y))+1
    sorts_o = torch.argsort(sftmax(x))+1
    sorts_factor = 1/sorts_t
    print(sorts_t)
    print(sorts_o)
    print(abs(sorts_t-sorts_o))
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)
    loss_ohe = loss*ohe
    loss_place = loss*abs(sorts_t-sorts_o)
    print(f"{sorts_t}\n{out}\n{ohe}\n{loss}\n{loss_ohe}")
    return (torch.mean(loss),torch.mean(loss_ohe), torch.mean(loss_place))

In [31]:
huber = nn.HuberLoss(reduction='none')

sorts_t = torch.argsort(sftmax(y))
out = sorts_t.narrow(1,0,3)
ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)

huber(sftmax(x),sftmax(y))*ohe

loss_customiser(huber, sftmax(x),sftmax(y))

tensor([[7, 2, 6, 3, 5, 4, 8, 1],
        [5, 2, 6, 7, 8, 3, 1, 4]])
tensor([[5, 2, 6, 7, 8, 3, 1, 4],
        [7, 2, 6, 3, 5, 4, 8, 1]])
tensor([[2, 0, 0, 4, 3, 1, 7, 3],
        [2, 0, 0, 4, 3, 1, 7, 3]])
tensor([[7, 2, 6, 3, 5, 4, 8, 1],
        [5, 2, 6, 7, 8, 3, 1, 4]])
tensor([[7, 2, 6],
        [5, 2, 6]])
tensor([[0, 0, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 0, 1, 1, 0]])
tensor([[7.9884e-02, 0.0000e+00, 2.7377e-03, 1.4947e-01, 4.7757e-04, 0.0000e+00,
         6.0555e-05, 2.0229e-02],
        [7.9884e-02, 0.0000e+00, 2.7377e-03, 1.4947e-01, 4.7757e-04, 0.0000e+00,
         6.0555e-05, 2.0229e-02]])
tensor([[0.0000e+00, 0.0000e+00, 2.7377e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.0555e-05, 2.0229e-02],
        [0.0000e+00, 0.0000e+00, 2.7377e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.0555e-05, 0.0000e+00]])


(tensor(0.0316), tensor(0.0016), tensor(0.1025))

In [ ]:
sftmax = torch.nn.Softmax(dim=1)
sftmax(y)

RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'